In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, GRU,Bidirectional,Dropout
from keras.utils import to_categorical
from tensorflow import keras

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Read the data

In [ ]:
train=pd.read_csv("/kaggle/input/ml-olympiad-sentiment-analysis-of-jumia-tunisia/train.csv",index_col='IdCustomer')
test=pd.read_csv("/kaggle/input/ml-olympiad-sentiment-analysis-of-jumia-tunisia/test.csv",index_col='IdCustomer')

In [ ]:
train

In [ ]:
test

# Tokenized text

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train['Text'])
X_text = tokenizer.texts_to_sequences(train['Text'])
max_length = max([len(sequence) for sequence in X_text])
X_text = pad_sequences(X_text, maxlen=max_length, padding='post')

### First we encode the target column with integer encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train['Target'] = label_encoder.fit_transform(train['Target'])


In [ ]:
train

# Try the RNN model

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_length))
model.add(Bidirectional(GRU(64, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))

optimizer = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

y_train = to_categorical(train['Target'])

model.fit(X_text, y_train, epochs=10, batch_size=32, validation_split=0.2)


In [ ]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_length))
model.add(Bidirectional(GRU(60, dropout=0.5, recurrent_dropout=0.2)))
model.add(Dense(30, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

optimizer = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

y_train = to_categorical(train['Target'])

model.fit(X_text, y_train, epochs=10, batch_size=32, validation_split=0.2)


In [ ]:
# Tokenize the text data
X_test = tokenizer.texts_to_sequences(test['Text'])

# Pad the sequences to have the same length
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')


In [ ]:



# Make predictions on the preprocessed test data
y_pred = model.predict(X_test)

# Convert the predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)
y_pred_labels = label_encoder.inverse_transform(y_pred_labels)

# Create a dataframe with the predicted class labels and the corresponding ids
pred_df = pd.DataFrame({'Target': y_pred_labels}, index=test.index)

# Save the predictions to a CSV file
pred_df.to_csv('predictions.csv')


In [ ]:
pred_df

In [ ]:
train